# OSS-DBS v2.0 Optimize Parameters
**First test version**

In [ ]:
from ngsolve.webgui import Draw as DrawNG
from ngsolve import TaskManager
from netgen.webgui import Draw
import nibabel as nib
import pandas as pd
import numpy as np
import logging
import ossdbs
import time

ossdbs.set_logger(logging.INFO)

In [ ]:
list_settings = []
list_parameters = []
impedance = []
vta_size = []
time_list = []
name = []

In [ ]:
parameters = {"MeshElementOrder": 2,
              "Type": "Default",
             }
list_parameters.append(parameters)
name.append("case_1")
###
parameters = {"MeshElementOrder": 3,
              "Type": "Default",
             }
list_parameters.append(parameters)
name.append("case_2")
###
parameters = {"MeshElementOrder": 3,
              "Type": "Fine",
             }
list_parameters.append(parameters)
name.append("case_3")

In [ ]:
for i in range(len(list_parameters)):
    
    settings = {
        "BrainRegion": {
            "Center": { "x[mm]": -9.48, "y[mm]": 11.61, "z[mm]": 4.68 },
            "Dimension": {"x[mm]": 40.0, "y[mm]": 40.0, "z[mm]": 40.0 },
            "Shape": "Ellipsoid"
        },
        "Electrodes":
        [
            {
            "Name": "BostonScientificVercise",
          "Rotation[Degrees]": 0.0,
          "Direction": { "x[mm]": 0.0, "y[mm]": 0.0, "z[mm]": 1.0 },
          "TipPosition": { "x[mm]": -9.48, "y[mm]": 11.61, "z[mm]": 4.68 },
    
          "Contacts": [
            {
              "Contact_ID": 1,
              "Active": True,
              "Current[A]": 0.0,
              "Voltage[V]": 1.0,
              "Floating": False,
              "MaxMeshSizeEdge": 0.01
            },
            {
              "Contact_ID": 2,
              "Active": True,
              "Current[A]": 0.0,
              "Voltage[V]": 0.0,
              "Floating": False,
              "MaxMeshSizeEdge": 0.01
            }
          ],
        "EncapsulationLayer": {
            "Thickness[mm]": 0.0,
            "Material": "Blood",
            "DielectricModel": "ColeCole4",
            "MaxMeshSize": 0.5
         }
        }
      ],
      "MaterialDistribution": {
        "MRIPath": "../../input_files/Butenko_segmask.nii.gz",
        "MRIMapping": {
          "Unknown": 0,
          "CSF": 1,
          "White matter": 2,
          "Gray matter": 3,
          "Blood": 4      
        },
        "DiffusionTensorActive": False,
        "DTIPath": ""
      },
    
      "DielectricModel": {
        "Type": "ColeCole4",
        "CustomParameters": None
      },
    
      "Mesh": {
        "LoadMesh": False,
        "MeshElementOrder": list_parameters[i]["MeshElementOrder"],
        "MeshingHypothesis": {
          "Type": list_parameters[i]["Type"],
          "MaxMeshSize": 0.1
        },
        "MeshSize":{
          "Edges": {},
          "Faces":{"E1C1": 0.1},
          "Volumes":{"Brain": 0.5}
        },
        "SaveMesh": False
      },
    
      "MeshRefinement": {
        "MeshSizeRatioContacts": 3.0
      },
    
      "FEMOrder": 2,
      "EQSMode": False, 
    
      "StimulationSignal": {
        "CurrentControlled": False,
        "Type": "Multisine",
        "ListOfFrequencies": [10000.0]
      },
        
        "Solver": 
        {
            "Type": "CG", 
            "Preconditioner": "bddc",
            "PreconditionerKwargs": {},
            "PrintRates": False,
            "MaximumSteps": 10000,
            "Precision": 1e-12
        },
        "PointModel": {
            "Pathway": {
                "Active": False, 
                "FileName": ""
            },
            "Lattice": {
                "Active": True,
                "Center": { "x[mm]": -9.48, "y[mm]": 11.62, "z[mm]": 4.72 },
                "Shape": { "x": 25, "y": 25, "z": 25},
                "Direction": {"x[mm]": 0, "y[mm]": 0, "z[mm]": 1},
                "PointDistance[mm]": 0.1,
            },
        },
    
      "OutputPath": "Results/",
      "ComputeImpedance": True,
      "TemplateSpace": False,
      "ActivationThresholdVTA": 0.2,
      "ExportVTK": True,
      "ExportElectrode": True
    }
    list_settings.append(settings)

In [ ]:
settings = list_settings[i]

## Prepeare Volume Conductor Model

In [ ]:
mri_image, dti_image = ossdbs.load_images(settings)
electrodes = ossdbs.generate_electrodes(settings)

region_parameters = settings["BrainRegion"]
brain_region = ossdbs.create_bounding_box(settings["BrainRegion"])
shape = settings["BrainRegion"]["Shape"]

brain = ossdbs.BrainGeometry(shape, brain_region)

model_geometry = ossdbs.ModelGeometry(brain, electrodes)

ossdbs.set_contact_and_encapsulation_layer_properties(settings, model_geometry)

dielectric_model = ossdbs.prepare_dielectric_properties(settings)

materials = settings["MaterialDistribution"]["MRIMapping"]
conductivity = ossdbs.ConductivityCF(mri_image,
                                  brain_region,
                                  dielectric_model,
                                  materials,
                                  model_geometry.encapsulation_layers,
                                  complex_data=settings["EQSMode"]
                                  )

## Run Volume Conductor Model
To run the Volume Conductor Model, the built-in TaskManager is use to parallize the computational heavy parts. During this step, the results will be stored in the specified folder.

In [ ]:
for idx in range(len(list_settings)):
    
    print('Run ', idx + 1, '/', len(list_settings))

    settings = list_settings[idx]
    
    time_0 = time.time()
    
    solver = ossdbs.prepare_solver(settings)
    
    with TaskManager():
        volume_conductor = ossdbs.prepare_volume_conductor_model(settings, model_geometry, conductivity, solver)
        ossdbs.run_volume_conductor_model(settings, volume_conductor)
        
    Z = volume_conductor.impedances
    
    impedance.append(Z)
    time_list.append(time.time() - time_0)

    print("Total impedance ", np.round(Z[0]))

    ###

    img = nib.load("Results/VTA_solution_WA.nii")

    affine = img.affine
    voxel_size = affine[0,0] * affine[1,1] * affine[2,2]
    
    data_shape = img.get_fdata().shape
    data = img.get_fdata()
    
    end_voxel = [data_shape[0], data_shape[1], data_shape[2]]
    
    counter = 0
    
    for i in range(0, data_shape[0]):
        for j in range(0, data_shape[1]):
            for k in range(0, data_shape[2]):
                if data[i,j,k] > settings["ActivationThresholdVTA"] :
                    counter = counter + 1

    vta_size.append(counter * voxel_size)

    print("VTA size: ", counter * voxel_size, "mm³")

In [ ]:
df = pd.DataFrame({'name': name,
                   'timing': time_list,
                   'impedance': impedance,
                   'parameters': list_parameters})

df.to_csv("Results/impedance_cases.csv", index=False)